# Faker 기반 가상데이터 생성기 안내

이 노트북은 JSON/CSV로 가상 데이터를 생성하는 과정을 단계별로 안내합니다. CSV 저장은 UTF-8-sig로 하여 Excel에서 바로 열기 편하도록 합니다.

## 데이터 스키마 개요

이 노트북은 이어드림 플랫폼의 데이터 스키마를 기반으로 가상 데이터를 생성합니다. 플랫폼은 재난 상황에서 대피소와 구호품 기부자를 연결하는 AI 기반 매칭 시스템입니다.

### 주요 기능
- 실시간 대피소 현황 모니터링
- AI 기반 구호품 수요 예측
- 스마트 매칭 알고리즘을 통한 기부자-대피소 연결
- 개인화된 추천 시스템
- 배송 및 물류 최적화

### AI/ML 모델 요구사항
- **다변량 LSTM 모델**: 대피소별 구호품 수요 예측
- **트위터 오픈소스 추천 알고리즘**: 사용자 맞춤형 기부 추천
- **실시간 분석**: 대피소 현황 및 긴급도 분석

### 핵심 데이터 테이블 구조

#### users 테이블
- 사용자 고유 ID, 이메일, 유형(공무원/일반), 이름, 전화번호, 주소, 선호 카테고리 등

#### shelters 테이블
- 대피소 ID, 관리자 ID, 이름, 재난 유형, 상태, 주소, 좌표, 수용 인원, 편의시설 등

#### relief_items 테이블
- 구호품 ID, 코드, 카테고리(식량/생활용품/의류/의약품), 소분류, 이름, 단위 등

#### user_donation_wishes 테이블
- 기부 의사 ID, 사용자 ID, 구호품 ID, 수량, 상태, 매칭 정보 등

#### shelter_relief_requests 테이블
- 요청 ID, 대피소 ID, 구호품 ID, 요청 수량, 현재 재고, 긴급도 등

#### donation_matches 테이블
- 매칭 ID, 기부 의사 ID, 요청 ID, 매칭 수량, 상태, 배송 정보 등

#### disaster_incidents 테이블
- 재난 사건 ID, 연도, 유형 코드, 지역, 피해 정보, 관련 대피소 등

#### consumption_info 테이블
- 소비 정보 ID, 대피소 ID, 재난 ID, 구호품 ID, 소비량, 기간, 만족도 등

### 데이터 관계도
```mermaid
graph TD
    A[users] --> B[user_donation_wishes]
    A --> C[shelters]
    C --> D[shelter_relief_requests]
    E[relief_items] --> B
    E --> D
    B --> F[donation_matches]
    D --> F
    G[disaster_incidents] -.-> C
    C --> L[consumption_info]
    G --> L
    E --> L
```

이 스키마를 기반으로 Faker(ko_KR)를 사용하여 가상 데이터를 생성합니다.

## 1) 필요한 라이브러리 불러오기

아래 셀에서 파이썬 표준라이브러리와 `pandas`, `faker` 등을 불러옵니다. 필요 시 pip로 설치하는 코드도 포함되어 있습니다.

In [1]:
# 셀 1: 필요한 패키지 설치(필요한 경우 주석 해제 후 실행)
# !pip install -r ../requirements.txt

import json
import os
from pathlib import Path
import subprocess

import pandas as pd
from faker import Faker

print('라이브러리 로드 완료')

라이브러리 로드 완료


## 2) 샘플 데이터 생성

딕셔너리와 리스트 구조의 간단한 샘플 데이터를 만듭니다. 실제 생성기와 동일한 필드 일부를 사용합니다.

In [ ]:
# 셀 2: 샘플 데이터 생성
fake = Faker('ko_KR')

users = []
for i in range(1, 6):
    users.append({
        'user_id': f'user_{i:04d}',
        'email': fake.safe_email(),
        'user_type': 'general_user',
        'name': fake.name(),
        'phone_number': fake.phone_number(),
    })

relief_items = [
    {'item_id': 'relief_item_001', 'item_name': '컵라면', 'category': '식량'},
    {'item_id': 'relief_item_002', 'item_name': '화장지', 'category': '생활용품'},
]

print('샘플 users:')
print(users)
print('\n샘플 relief_items:')
print(relief_items)

## 3) JSON 생성기 실행 (generate_fake_data.py)

`tools/generate_fake_data.py`는 Faker(ko_KR)를 사용하여 데이터 스키마에 맞춘 가상 데이터를 생성하고 JSON 파일로 저장합니다. 주요 기능:

- 사용자, 대피소, 구호품, 기부 의사, 요청, 매칭, 재난 사건, 소비 정보 테이블 생성
- 한국어 로케일 사용
- 대한민국 범위 내 랜덤 좌표 생성
- ISO 형식 날짜/시간 사용

아래에서 실제 스크립트를 실행합니다.

In [ ]:
# 셀 3: JSON 생성기 실행
# 작은 샘플로 실행 (실제로는 명령행에서 --users 등 옵션 조정 가능)
import subprocess

result = subprocess.run(['python', '../generate_fake_data.py', '--seed', '42', '--users', '10', '--shelters', '5', '--relief_items', '8', '--wishes', '15', '--requests', '12', '--matches', '10', '--incidents', '3', '--consumptions', '8'], capture_output=True, text=True, cwd='.')

print('실행 결과:')
print(result.stdout)
if result.stderr:
    print('에러:', result.stderr)

## 4) CSV 생성기 실행 (generate_fake_data_csv.py)

`tools/generate_fake_data_csv.py`는 `generate_fake_data.py`의 생성 함수를 재사용하여 데이터를 CSV로 저장합니다. 특징:

- pandas를 사용하여 DataFrame 변환
- UTF-8-sig 인코딩으로 Excel에서 바로 열기 가능
- 동일한 데이터 구조 유지

아래에서 실제 스크립트를 실행합니다.

In [5]:
# 셀 4: CSV 생성기 실행
# 동일한 시드로 실행하여 JSON과 동일한 데이터 생성
result = subprocess.run(['python', '../generate_fake_data_csv.py', '--seed', '42', '--users', '10', '--shelters', '5', '--relief_items', '8', '--wishes', '15', '--requests', '12', '--matches', '10', '--incidents', '3', '--consumptions', '8'], capture_output=True, text=True, cwd='.')

print('실행 결과:')
print(result.stdout)
if result.stderr:
    print('에러:', result.stderr)

실행 결과:
CSV 생성 완료: output_csv 폴더에 CSV 파일들을 생성했습니다.



## 5) 생성된 파일 확인

위에서 실행한 스크립트로 생성된 파일들을 확인합니다.

## 6) 생성 파일 읽기 및 검증

생성된 JSON과 CSV 파일을 읽어 샘플 데이터를 확인합니다.

In [ ]:
# 셀 6: 생성된 파일 읽기 및 검증
import json

# JSON 파일 읽기 예시
json_path = Path('../output/users.json')
if json_path.exists():
    with json_path.open('r', encoding='utf-8') as f:
        users = json.load(f)
    print('users.json 샘플 (처음 3개):')
    for user in users[:3]:
        print(user)
    print(f'총 레코드 수: {len(users)}')
else:
    print('users.json이 없습니다.')

# CSV 파일 읽기 예시
csv_path = Path('../output_csv/users.csv')
if csv_path.exists():
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    print('\nusers.csv 샘플:')
    print(df.head())
    print(f'총 행 수: {len(df)}')
else:
    print('users.csv가 없습니다.')

# 다른 파일들도 확인
print('\n생성된 파일 목록:')
for folder in ['../output', '../output_csv']:
    if Path(folder).exists():
        files = list(Path(folder).glob('*.json')) + list(Path(folder).glob('*.csv'))
        for f in files:
            print(f'- {f}')